In [1]:
from pyrootmemo.waldron import Waldron
from pyrootmemo.materials import MultipleRoots, Soil, Shearzone, Interface
from pyrootmemo.tools.helpers import units
import numpy as np

In [2]:
# generate some root data
from collections import namedtuple
Parameter = namedtuple("parameter", "value unit")
roots = MultipleRoots(
    species = "test_species",
    diameter = Parameter([1, 2, 1, 3, 4, 5, 6, 3], 'mm'),
    tensile_strength = Parameter([40, 30, 40, 20, 10, 5, 20, 10], 'MPa'),
    elastic_modulus = Parameter([100, 100, 100, 100, 100, 100, 20, 100], 'MPa'),
    length = Parameter([1, 1, 1, 1, 1, 1, 1, 1], 'm'),
    #azimuth_angle = Parameter([0, 0, 0, 0, 0, 0, 0, 0], 'degrees'),
    #elevation_angle = Parameter([-40, -30, -20, -10, 0, 10, 20, 30], 'degrees')
)
# generate some soil data
soil = Soil(
    "Sand",
    cohesion = Parameter(2, "kPa"),
    friction_angle = Parameter(30, "degrees")
)
# shearzone object
shearzone = Shearzone(
    area = Parameter(1, "m^2"),
    thickness = Parameter(10, "mm"),
    effective_stress = Parameter(20, "kPa")
)
# soil--root interface
interface = Interface(
    shear_strength = Parameter(10, "kPa")
)

In [3]:
# Create Waldron object
waldron = Waldron(shearzone, roots, soil, interface)

In [7]:
waldron.reinforcement(0.1 * units('mm'))

ValueError: operands could not be broadcast together with shapes (8,) (8,2) 

In [5]:
print(roots.initial_orientation_vector())
print(roots.initial_orientation_vector(axis_angle = np.array([0.1, 0.1, 0.1])))

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[ 0.10448826 -0.09451324  0.99002498]
 [ 0.10448826 -0.09451324  0.99002498]
 [ 0.10448826 -0.09451324  0.99002498]
 [ 0.10448826 -0.09451324  0.99002498]
 [ 0.10448826 -0.09451324  0.99002498]
 [ 0.10448826 -0.09451324  0.99002498]
 [ 0.10448826 -0.09451324  0.99002498]
 [ 0.10448826 -0.09451324  0.99002498]]


In [6]:
import numpy as np
a = np.random.rand(5, 3)
b = np.random.rand(3, 3)
print(np.tensordot(a, b, [-1, -1]))

[[0.79248961 1.26777654 1.67290478]
 [0.55430229 1.17710956 1.6713217 ]
 [0.85346889 1.23233797 1.57315743]
 [0.66721294 1.24711776 1.7050247 ]
 [0.86166668 1.28238555 1.65555142]]
